In [2]:
import pandas as pd

In [ ]:
data = {'Platoon': ['A','A','A','A','A','A','B','B','B','B','B','C','C','C','C','C'],
        'Date' : [1,2,3,4,5,6,1,2,3,4,5,1,2,3,4,5],
       'Casualties': [1,4,5,7,5,5,6,1,4,5,6,7,4,6,4,6]}
df = pd.DataFrame(data)
df

In [23]:
df.groupby(['Platoon'])['Casualties'].apply(lambda x: x.rolling(5, 1).mean().shift().bfill())

0     1.000000
1     1.000000
2     2.500000
3     3.333333
4     4.250000
5     4.400000
6     6.000000
7     6.000000
8     3.500000
9     3.666667
10    4.000000
11    7.000000
12    7.000000
13    5.500000
14    5.666667
15    5.250000
Name: Casualties, dtype: float64

Using this notebook for working changes before putting them in a python file

In [1]:
import database_utility
import baseball_models
import pandas as pd

In [2]:
sqlite_file = 'baseball_data.sqlite'
dh = database_utility.DatabaseHelper(sqlite_file)
#This will give a ValueError when using pd.read_json if the path doesn't exist
filepath = r'C:\Users\Chris\Google Drive\Programming\Baseball\Web Scraping\bbref_scraper\bbref_scraper\bbref.jl'

In [3]:
pitching_df = dh.calc_pitching_fd_score(filepath)

C:\Users\Chris\Google Drive\Programming\Baseball\database_utility.py:181: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  qual_start_df['quality_start'] = 1


In [6]:
selector = baseball_models.BaseballModelSelection()
X, y, tscv = selector.train_test_split(pitching_df, num_splits=2)

In [7]:
tscv.split(X)

<generator object TimeSeriesSplit.split at 0x000002E9108A0830>

In [8]:
for train_index, test_index in tscv.split(X):
    print("TRAIN:", train_index, "TEST:", test_index)

TRAIN: [   0    1    2 ... 1526 1527 1528] TEST: [1529 1530 1531 ... 3053 3054 3055]
TRAIN: [   0    1    2 ... 3053 3054 3055] TEST: [3056 3057 3058 ... 4580 4581 4582]


In [66]:
X['fd_score'] = y

#X['avg']  = X.groupby(['player', 'game_date', 'game_id'])['fd_score']

In [74]:
game_to_predict = '2017-08-24'
player = 'alberma01'

In [8]:
filtered_df = mov_df[(mov_df['player'] == player) & (mov_df['game_date'] < game_to_predict)]

filtered_df.tail(1)['moving_avg']

NameError: name 'mov_df' is not defined

In [26]:
X.head(20)

,game_id,away_team,home_team,game_date,start_time,attendance,game_situation,player,team,IP,...,wpa_def,leverage_index_avg,re24_def,position,Day of Week,year,stadium,win_recorded,is_first_pitcher,quality_start
0,2015-04-06Marlins Park4:22 p.m.teherju01,Atlanta Braves,Miami Marlins,2015-04-06,4:22 p.m.,"36,969","Day Game, on grass",teherju01,away,6.0,...,-0.118,1.27,-0.1,", W (1-0)",Monday,2015.0,Marlins Park,1.0,1.0,1.0
1,2015-04-06Marlins Park4:22 p.m.johnsji04,Atlanta Braves,Miami Marlins,2015-04-06,4:22 p.m.,"36,969","Day Game, on grass",johnsji04,away,1.1,...,0.263,2.50,1.0,", H (1)",Monday,2015.0,Marlins Park,0.0,0.0,0.0
2,2015-04-06Marlins Park4:22 p.m.alvarhe01,Atlanta Braves,Miami Marlins,2015-04-06,4:22 p.m.,"36,969","Day Game, on grass",alvarhe01,home,7.0,...,0.066,1.05,1.1,", L (0-1)",Monday,2015.0,Marlins Park,0.0,0.0,0.0
3,2015-04-06Marlins Park4:22 p.m.ramosaj01,Atlanta Braves,Miami Marlins,2015-04-06,4:22 p.m.,"36,969","Day Game, on grass",ramosaj01,home,1.0,...,0.032,0.47,0.4,NA,Monday,2015.0,Marlins Park,0.0,0.0,0.0
4,2015-04-06Marlins Park4:22 p.m.avilalu01,Atlanta Braves,Miami Marlins,2015-04-06,4:22 p.m.,"36,969","Day Game, on grass",avilalu01,away,0.2,...,0.317,4.16,1.7,", H (1)",Monday,2015.0,Marlins Park,0.0,0.0,0.0
5,2015-04-06Marlins Park4:22 p.m.grillja01,Atlanta Braves,Miami Marlins,2015-04-06,4:22 p.m.,"36,969","Day Game, on grass",grillja01,away,1.0,...,0.181,2.47,0.4,", S (1)",Monday,2015.0,Marlins Park,0.0,0.0,0.0
6,2015-04-06Marlins Park4:22 p.m.dunnmi01,Atlanta Braves,Miami Marlins,2015-04-06,4:22 p.m.,"36,969","Day Game, on grass",dunnmi01,home,1.0,...,0.045,0.61,0.4,NA,Monday,2015.0,Marlins Park,0.0,0.0,0.0
7,2016-04-05Marlins Park7:37 p.m.verlaju01,Detroit Tigers,Miami Marlins,2016-04-05,7:37 p.m.,"36,911","Night Game, on grass",verlaju01,away,6.0,...,0.057,0.61,-0.2,NA,Tuesday,2016.0,Marlins Park,0.0,0.0,0.0
8,2016-04-05Marlins Park7:37 p.m.lowema01,Detroit Tigers,Miami Marlins,2016-04-05,7:37 p.m.,"36,911","Night Game, on grass",lowema01,away,1.0,...,-0.004,1.98,-0.5,", H (1)",Tuesday,2016.0,Marlins Park,0.0,0.0,0.0
9,2016-04-05Marlins Park7:37 p.m.verhadr01,Detroit Tigers,Miami Marlins,2016-04-05,7:37 p.m.,"36,911","Night Game, on grass",verhadr01,away,1.0,...,0.129,3.28,0.5,", W (1-0)",Tuesday,2016.0,Marlins Park,1.0,0.0,0.0


In [25]:
selector = baseball_models.BaseballModelSelection()
mov_avg = baseball_models.MovingAverageModel()

X, y, tscv = selector.train_test_split(pitching_df, num_splits = 5)

#X_train = X[X['game_date'] <= '2018-05-31']
#y_train = y.iloc[:len(X_train)]

#X_test = X[X['game_date'] > '2018-05-31']
#y_test = y.iloc[len(X_train):]

i = 0
for train_index, test_index in tscv.split(X):
    if i < 4:
        print("TRAIN:", train_index, "TEST:", test_index)
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]
        i += 1

TRAIN: [  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17
  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35
  36  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53
  54  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71
  72  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89
  90  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107
 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125
 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143
 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161
 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179
 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197
 198 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215
 216 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233
 234 235 236 237 238 239 240 241 242 243 244

In [ ]:
#fit the moving average model with all the data just for fun
mov_avg.fit(X_train, y_train)

In [6]:
predictions = mov_avg.predict(X_test)

In [23]:
import numpy as np

errors = []

for i,val in enumerate(predictions):
    errors.append(val - y_test.iloc[i])